Import libraries

In [1]:
#!conda install -c conda-forge request --yes
#import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner

#!conda install -c conda-forge geopy --yes 
#%pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage

#%pip install sklearn
from sklearn.cluster import KMeans
import json # library to handle JSON files

#%pip install geopandas
#!conda install -c conda-forge geopandas --yes
import geopandas as gdp

#!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

import requests

print('Libraries imported.')

Libraries imported.


Set env variables

In [3]:
CLIENT_ID = '2LKE1PE5RDDKHGJLBMTQDMDL5QP1RBIDI3HZLBQTY25I2TKV' # your Foursquare ID
CLIENT_SECRET = 'RXJD0BEBX4SGH13YPQEVC4C1VGUUAVBC2DDXFXRC3ZII1CTG' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT=100 # Set limits

Import and filter shopping places' prices - data wrangling

In [310]:
prices=pd.read_csv('/Users/matfarina/Desktop/GitHub_Projects/coursera_capstone_project/valori.csv', "r", delimiter = ";", header=1, skiprows=0)
prices=prices[(prices["Descr_Tipologia"]=="Negozi")]
prices=prices[["Zona", "Compr_min", "Compr_max"]].reset_index(drop=True)
prices["Average price"]=prices[['Compr_min', 'Compr_max']].mean(axis=1)
prices=prices.groupby("Zona").mean().reset_index().sort_values(by=["Average price"], ascending=False)

Find coordinates and define map

In [5]:
address = 'Milan, Italy'

geolocator = Nominatim(user_agent="milan_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))

The geograpical coordinate of Milan, Italy are 45.4668, 9.1905.


In [55]:
map_milan = folium.Map(location=[latitude, longitude], zoom_start=12)

Import Milan boundaries

In [40]:
with open('/Users/matfarina/Desktop/GitHub_Projects/coursera_capstone_project/Comune_Milano_Coordinates.geojson', "r") as json_data:
    milan_geo_data = json.load(json_data)

Plot map with the prices on it

In [56]:
choropleth = folium.Choropleth(    
    geo_data=milan_geo_data,
    data=prices,
    columns=['Zona', 'Average price'],
    key_on='feature.properties.CODZONA',
    fill_color='YlOrRd', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='Shops Square meter price')

choropleth = choropleth.add_to(map_milan)

map_milan

Find center of areas

In [157]:
milan_coor=gdp.GeoDataFrame.from_features(milan_geo_data)
milan_coor['centroids_long']=milan_coor["geometry"].centroid.x
milan_coor['centroids_lat']=milan_coor["geometry"].centroid.y
area_centers = milan_coor[["CODZONA", "centroids_long", "centroids_lat"]]
area_centers = area_centers.rename(columns={"centroids_long":"Longitude", "centroids_lat":"Latitude"})

Import venues around the center of the cities area

In [318]:
area_centers

,CODZONA,Longitude,Latitude
0,R2,9.205252,45.409661
1,D25,9.115738,45.456855
2,D13,9.249946,45.479850
3,B17,9.173162,45.472447
4,D30,9.124105,45.505802
5,D24,9.138160,45.462397
6,B18,9.191606,45.475631
7,D12,9.227549,45.473697
8,B21,9.172414,45.458709
9,D36,9.203340,45.500451


In [160]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


In [165]:
milan_venues = getNearbyVenues(names=area_centers['CODZONA'],latitudes=area_centers['Latitude'],longitudes=area_centers['Longitude'])
print("Done!")

Done!


Get most common venues per area

In [200]:
# one hot encoding
milan_onehot = pd.get_dummies(milan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
milan_onehot['Neighborhood'] = milan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [milan_onehot.columns[-1]] + list(milan_onehot.columns[:-1])
milan_onehot = milan_onehot[fixed_columns]


In [201]:
milan_grouped = milan_onehot.groupby('Neighborhood').mean().reset_index()

In [202]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [274]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = milan_grouped['Neighborhood']

for ind in np.arange(milan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(milan_grouped.iloc[ind, :], num_top_venues)


Data K-cluster

In [263]:
# set number of clusters
kclusters = 5

milan_grouped_clustering = milan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(milan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([4, 0, 0, 0, 0, 0, 1, 1, 1, 0], dtype=int32)

In [311]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

milan_merged = area_centers

# merge useful data in a single dataframe
milan_merged = milan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='CODZONA').dropna()
milan_merged = milan_merged.join(prices.set_index('Zona'), on='CODZONA').dropna()


Plot the venues

In [312]:
# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(milan_merged['Latitude'], milan_merged['Longitude'], milan_merged['CODZONA'], milan_merged['Cluster Labels'].astype("int")):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_milan)

In [278]:
map_milan

Data Analysis

In [313]:
milan_merged.loc[milan_merged['Cluster Labels'] == 0, milan_merged.columns[[0] + list(range(4, milan_merged.shape[1]-3))+[-1]]]

,CODZONA,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Average price
3,B17,Italian Restaurant,Cocktail Bar,Platform,Nightclub,Burger Joint,Plaza,Café,Tram Station,Coffee Shop,Japanese Restaurant,5150.0
6,B18,Italian Restaurant,Japanese Restaurant,Ice Cream Shop,Coffee Shop,Hotel,Restaurant,Sandwich Place,Ramen Restaurant,Art Museum,Wine Bar,6050.0
12,D16,Italian Restaurant,Tram Station,Pizza Place,Sushi Restaurant,German Restaurant,Gym,Middle Eastern Restaurant,Gourmet Shop,Burger Joint,Nightclub,1925.0
13,C17,Italian Restaurant,Plaza,Sushi Restaurant,Asian Restaurant,Ice Cream Shop,Sandwich Place,Pizza Place,Burger Joint,Supermarket,Cocktail Bar,3875.0
15,C18,Italian Restaurant,Cocktail Bar,Art Gallery,Pizza Place,Seafood Restaurant,Bistro,Café,Trattoria/Osteria,Ice Cream Shop,Burger Joint,3675.0
18,B13,Italian Restaurant,Hotel,Café,Bakery,Bistro,Pizza Place,Japanese Restaurant,Nightclub,Plaza,Restaurant,5425.0
19,C15,Italian Restaurant,Café,Bakery,Burger Joint,Hotel,Sushi Restaurant,Breakfast Spot,Gym / Fitness Center,Government Building,Noodle House,2662.5
20,B16,Italian Restaurant,Café,Plaza,Ice Cream Shop,Theater,Wine Bar,Sandwich Place,Gift Shop,Dessert Shop,Art Gallery,5750.0
23,B15,Italian Restaurant,Hotel,Ice Cream Shop,Boutique,Plaza,Sandwich Place,Café,Art Museum,Dessert Shop,Wine Bar,6500.0
24,C12,Italian Restaurant,Cocktail Bar,Hotel,Pizza Place,Café,Chinese Restaurant,Ice Cream Shop,Restaurant,Bar,Japanese Restaurant,5050.0


In [314]:
milan_merged.loc[milan_merged['Cluster Labels'] == 1, milan_merged.columns[[0] + list(range(4, milan_merged.shape[1]-3))+[-1]]]

,CODZONA,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Average price
1,D25,Bus Stop,Pizza Place,Metro Station,Italian Restaurant,Dance Studio,Gym,Bus Station,Gastropub,Gay Bar,Food Truck,1325.0
4,D30,Café,Pizza Place,Supermarket,Coffee Shop,Tram Station,Seafood Restaurant,Farmers Market,Fast Food Restaurant,General Entertainment,Gay Bar,1837.5
5,D24,Pizza Place,Café,Plaza,Italian Restaurant,Restaurant,Sushi Restaurant,Ice Cream Shop,Hotel,Diner,Pharmacy,2025.0
7,D12,Pizza Place,Café,Hotel,Seafood Restaurant,Ice Cream Shop,Tram Station,Plaza,Japanese Restaurant,Pub,Art Gallery,2537.5
8,B21,Italian Restaurant,Pizza Place,Café,Seafood Restaurant,Pub,Ice Cream Shop,Japanese Restaurant,Plaza,Bistro,Supermarket,3937.5
9,D36,Seafood Restaurant,Hotel,Italian Restaurant,Chinese Restaurant,Pharmacy,Bed & Breakfast,Café,Fast Food Restaurant,Food Truck,General Entertainment,1750.0
14,B19,Pizza Place,Italian Restaurant,Furniture / Home Store,Ice Cream Shop,Cocktail Bar,Hotel,Sandwich Place,Spanish Restaurant,Sporting Goods Shop,Café,5250.0
17,C20,Pizza Place,Italian Restaurant,Ice Cream Shop,Seafood Restaurant,Bakery,Café,Piadineria,Boutique,Flower Shop,Greek Restaurant,3187.5
21,B20,Hotel,Café,Pizza Place,Burger Joint,Dessert Shop,Tram Station,Plaza,Restaurant,Salad Place,Sushi Restaurant,3925.0
27,C16,Hotel,Italian Restaurant,Café,Pizza Place,Tram Station,Sushi Restaurant,Japanese Restaurant,Bed & Breakfast,Seafood Restaurant,Nightclub,3750.0


In [315]:
milan_merged.loc[milan_merged['Cluster Labels'] == 2, milan_merged.columns[[0] + list(range(4, milan_merged.shape[1]-3))+[-1]]]

,CODZONA,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Average price
25,E6,Electronics Store,Women's Store,Food Court,Gift Shop,German Restaurant,General Entertainment,Gay Bar,Gastropub,Gaming Cafe,Furniture / Home Store,1175.0


In [317]:
milan_merged.loc[milan_merged['Cluster Labels'] == 3, milan_merged.columns[[0] + list(range(4, milan_merged.shape[1]-3))+[-1]]]

,CODZONA,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Average price
10,E7,Dog Run,Playground,Gym,Historic Site,Bus Stop,Farm,Farmers Market,German Restaurant,General Entertainment,Gay Bar,1000.0


In [316]:
milan_merged.loc[milan_merged['Cluster Labels'] == 4, milan_merged.columns[[0] + list(range(4, milan_merged.shape[1]-3))+[-1]]]

,CODZONA,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Average price
2,D13,Theater,Plaza,Buffet,Steakhouse,Supermarket,Café,Nightclub,Electronics Store,Farmers Market,Fast Food Restaurant,1725.0
16,E8,Sports Club,Park,Plaza,Playground,Bar,Historic Site,Food Court,General Entertainment,Gay Bar,Gastropub,1225.0
22,B12,Boutique,Plaza,Italian Restaurant,Hotel,Ice Cream Shop,Monument / Landmark,Clothing Store,Coffee Shop,Dessert Shop,Art Museum,14475.0
30,D21,Theater,Farm,Soccer Field,Bakery,Trattoria/Osteria,Café,Farmers Market,Fast Food Restaurant,German Restaurant,General Entertainment,1575.0
31,D32,Café,Ice Cream Shop,General Entertainment,Park,Hostel,Convenience Store,Supermarket,Cocktail Bar,Health Food Store,Gym / Fitness Center,1625.0
32,D34,Café,Italian Restaurant,Plaza,Steakhouse,Piadineria,Sandwich Place,Sushi Restaurant,Multiplex,Shopping Mall,Gym,1475.0
37,D10,Dog Run,Park,Italian Restaurant,Soccer Field,Performing Arts Venue,Bookstore,Food Court,Gay Bar,Gastropub,Gaming Cafe,1900.0
38,D28,Nightclub,Racecourse,Bed & Breakfast,Outdoor Sculpture,Cocktail Bar,Golf Course,Burger Joint,Grocery Store,Seafood Restaurant,Optical Shop,1250.0
